# Blackjack AR

In [1]:
from __future__ import print_function       # Python 2/3 compatibility
import cv2                                  # Import the OpenCV library
import mediapipe as mp                      # Import Mediapipe
import numpy as np                          # Import Numpy library

from enum import Enum
from ast import arg
import sys
import time
import random
from enum import Enum
from typing import List, Tuple, Optional

In [2]:
class BlackjackAction(Enum):
    NONE = 0
    HIT = 1
    STAND = 2
    DOUBLE = 3
    SPLIT = 4

class GestureRecognizer:
    def __init__(self):
        self.mp_hands = mp.solutions.hands
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        
    def detect_gesture(self, hand_landmarks):
        """
        Detecta el gesto de la mano y retorna la acción correspondiente de blackjack
        """
        total_fingers, fingers_up, is_right_hand = self.count_fingers_up(hand_landmarks)
        
        # Detectar gestos específicos para blackjack
        if self._is_hit_gesture(total_fingers, fingers_up):
            return BlackjackAction.HIT
        elif self._is_stand_gesture(total_fingers, fingers_up):
            return BlackjackAction.STAND
        elif self._is_double_gesture(total_fingers, fingers_up):
            return BlackjackAction.DOUBLE
        elif self._is_split_gesture(total_fingers, fingers_up):
            return BlackjackAction.SPLIT
        
        return BlackjackAction.NONE

    def _is_hit_gesture(self, total_fingers, fingers_up):
        """Un dedo levantado (índice) para pedir carta"""
        return total_fingers == 1 and fingers_up[1]

    def _is_stand_gesture(self, total_fingers, fingers_up):
        """Puño cerrado para plantarse"""
        return total_fingers == 0 or (total_fingers == 1 and fingers_up[0] == 1)

    def _is_double_gesture(self, total_fingers, fingers_up):
        """Dos dedos levantados (índice y medio) para doblar"""
        return total_fingers == 2 and fingers_up[1] and fingers_up[2]
    
    def _is_split_gesture(self, total_fingers, fingers_up):
        """Dos dedos levantados (pulgar e índice) para dividir las cartas"""
        return total_fingers == 2 and fingers_up[0] and fingers_up[1]

    def count_fingers_up(self, hand_landmarks):
        """
        Cuenta dedos levantados y determina su posición
        """
        finger_tips = [4, 8, 12, 16, 20]
        finger_bases = [2, 5, 9, 13, 17]
        fingers_up = []

        # Detectar mano derecha/izquierda
        is_right_hand = hand_landmarks.landmark[2].x < hand_landmarks.landmark[0].x

        # Comprobar pulgar
        thumb_tip_x = hand_landmarks.landmark[4].x
        thumb_ip_x = hand_landmarks.landmark[3].x
        fingers_up.append(thumb_tip_x < thumb_ip_x if is_right_hand else thumb_tip_x > thumb_ip_x)

        # Comprobar resto de dedos
        for tip, base in zip(finger_tips[1:], finger_bases[1:]):
            fingers_up.append(hand_landmarks.landmark[tip].y < hand_landmarks.landmark[base].y)

        return sum(fingers_up), fingers_up, is_right_hand

    def draw_debug_info(self, frame, action, hand_landmarks, fingers_up, is_right_hand):
        """
        Dibuja información de depuración en el frame
        """
        # Dibujar landmarks de la mano
        self.mp_drawing.draw_landmarks(
            frame,
            hand_landmarks,
            self.mp_hands.HAND_CONNECTIONS,
            self.mp_drawing_styles.get_default_hand_landmarks_style(),
            self.mp_drawing_styles.get_default_hand_connections_style()
        )

        # Mostrar acción detectada
        action_text = f"Action: {action.name}"
        cv2.putText(frame, action_text, (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Mostrar estado de los dedos
        finger_names = ["Thumb", "Index", "Middle", "Ring", "Pinky"]
        for i, (finger, is_up) in enumerate(zip(finger_names, fingers_up)):
            status = "Up" if is_up else "Down"
            cv2.putText(frame, f"{finger}: {status}", (10, 60 + (30 * i)),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)
            
        cv2.putText(frame, f"Mano derecha: {is_right_hand}", (10, 210),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

In [14]:
class Card:
    def __init__(self, suit, value, image):
        self.suit = suit
        self.value = value
        self.image = image
        self.isVisible = True

class GameState(Enum):
    WAITING_FOR_BET = 0
    PLAYER_TURN = 1
    DEALER_TURN = 2
    GAME_OVER = 3

class BlackjackGame:
    def __init__(self):
        self.deck = []
        self.player_hands = [[]]  # Lista de manos para permitir splits
        self.dealer_hand = []
        self.current_hand_index = 0
        self.player_chips = 2000
        self.current_bet = 0
        self.game_state = GameState.WAITING_FOR_BET
        self.initialize_deck()

    def initialize_deck(self):
        """Inicializa el mazo de cartas"""
        suits = ['clubs', 'diamonds', 'hearts', 'spades']
        values = ['2', '3', '4', '5', '6', '7', '8', '9', '10', 'jack', 'queen', 'king', 'ace']
        self.deck = [Card(suit, value, f'cartasPoker/{value}_of_{suit}.png') for suit in suits for value in values]
        random.shuffle(self.deck)

    def get_game_state(self) -> dict:
        """Retorna el estado actual del juego"""
        return {
            'player_hands': self.player_hands,
            'dealer_hand': self.dealer_hand,
            'current_hand_index': self.current_hand_index,
            'player_chips': self.player_chips,
            'current_bet': self.current_bet,
            'game_state': self.game_state,
            'can_split': self.can_split() if self.game_state == GameState.PLAYER_TURN else False
        }

    def can_split(self) -> bool:
        """Verifica si el jugador puede dividir su mano"""
        if (self.game_state != GameState.PLAYER_TURN or 
            self.current_hand_index >= len(self.player_hands)):
            return False
            
        current_hand = self.player_hands[self.current_hand_index]
        check_first_card = current_hand[0].value == '10' or current_hand[0].value == 'jack' or current_hand[0].value == 'queen' or current_hand[0].value == 'king'
        check_second_card = current_hand[1].value == '10' or current_hand[1].value == 'jack' or current_hand[1].value == 'queen' or current_hand[1].value == 'king'

        return (len(current_hand) == 2 and 
                check_first_card and check_second_card and 
                len(self.player_hands) < 4 and 
                self.player_chips >= self.current_bet)


    def process_action(self, action: BlackjackAction) -> Tuple[bool, str]:
        """Procesa una acción del jugador y retorna (éxito, mensaje)"""
        if self.game_state != GameState.PLAYER_TURN:
            return False, "No es tu turno"

        if self.current_hand_index >= len(self.player_hands):
            self.dealer_turn()
            return False, "Turno del dealer"

        current_hand = self.player_hands[self.current_hand_index]

        if action == BlackjackAction.HIT:
            current_hand.append(self.deck.pop())
            if self.calculate_hand_value(current_hand) > 21:
                self.next_hand()
            return True, "Carta repartida"

        elif action == BlackjackAction.STAND:
            self.next_hand()
            return True, "Te plantas"

        elif action == BlackjackAction.DOUBLE:
            if len(current_hand) != 2 or self.player_chips < self.current_bet:
                return False, "No puedes doblar"
            self.player_chips -= self.current_bet
            self.current_bet *= 2
            current_hand.append(self.deck.pop())
            self.next_hand()
            return True, "Apuesta doblada"

        elif action == BlackjackAction.SPLIT:
            if not self.can_split():
                return False, "No puedes dividir"
            self.player_chips -= self.current_bet
            new_hand = [current_hand.pop()]
            new_hand.append(self.deck.pop())
            current_hand.append(self.deck.pop())
            self.player_hands.append(new_hand)
            return True, "Mano dividida"

        return False, "Acción no válida"

    def next_hand(self):
        """Pasa a la siguiente mano o al turno del dealer"""
        self.current_hand_index += 1
        if self.current_hand_index >= len(self.player_hands):
            self.dealer_turn()

    def dealer_turn(self):
        """Ejecuta el turno del dealer"""
        self.game_state = GameState.DEALER_TURN
        self.dealer_hand[1].isVisible = True

        hands_over_limit = 0
        for hand in self.player_hands:
            if self.calculate_hand_value(hand) > 21:
                hands_over_limit += 1
        if hands_over_limit == len(self.player_hands):
            self.end_game()
            return

        while self.calculate_hand_value(self.dealer_hand) < 17:
            self.dealer_hand.append(self.deck.pop())
        self.end_game()

    def calculate_hand_value(self, hand: List[Card]) -> int:
        """Calcula el valor de una mano teniendo en cuenta los ases"""
        value = 0
        aces = 0
        
        for card in hand:
            if card.value in ['jack', 'queen', 'king']:
                value += 10
            elif card.value == 'ace':
                aces += 1
            else:
                value += int(card.value)
        
        for _ in range(aces):
            if value + 11 <= 21:
                value += 11
            else:
                value += 1
                
        return value

    def deal_initial_cards(self):
        """Reparte las cartas iniciales"""
        self.player_hands = [[self.deck.pop(), self.deck.pop()]]
        self.dealer_hand = [self.deck.pop(), self.deck.pop()]
        self.dealer_hand[1].isVisible = False
        self.current_hand_index = 0
        self.game_state = GameState.PLAYER_TURN

    def place_bet(self, amount: int) -> bool:
        """Coloca una apuesta"""
        if self.game_state != GameState.WAITING_FOR_BET or amount > self.player_chips:
            return False
        self.current_bet = amount
        self.player_chips -= amount
        self.deal_initial_cards()
        return True

    def end_game(self):
        """Finaliza el juego y calcula las ganancias"""
        self.game_state = GameState.GAME_OVER
        dealer_value = self.calculate_hand_value(self.dealer_hand)
        dealer_bust = dealer_value > 21

        for hand in self.player_hands:
            player_value = self.calculate_hand_value(hand)
            if player_value > 21:
                continue  # Mano perdida
            elif dealer_bust or player_value > dealer_value:
                if player_value == 21 and len(hand) == 2:
                    self.player_chips += self.current_bet * 2 + self.current_bet // 2
                self.player_chips += self.current_bet * 2   # Mano ganada
            elif player_value == dealer_value:
                self.player_chips += self.current_bet  # Empate
        

In [10]:
class DeckShowAruco:

    def __init__(self):
        self.desired_aruco_dictionary = "DICT_4X4_100"
        self.ARUCO_DICT = {
            "DICT_4X4_50": cv2.aruco.DICT_4X4_50,
            "DICT_4X4_100": cv2.aruco.DICT_4X4_100,
            "DICT_4X4_250": cv2.aruco.DICT_4X4_250,
            "DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
            "DICT_5X5_50": cv2.aruco.DICT_5X5_50,
            "DICT_5X5_100": cv2.aruco.DICT_5X5_100,
            "DICT_5X5_250": cv2.aruco.DICT_5X5_250,
            "DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
            "DICT_6X6_50": cv2.aruco.DICT_6X6_50,
            "DICT_6X6_100": cv2.aruco.DICT_6X6_100,
            "DICT_6X6_250": cv2.aruco.DICT_6X6_250,
            "DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
            "DICT_7X7_50": cv2.aruco.DICT_7X7_50,
            "DICT_7X7_100": cv2.aruco.DICT_7X7_100,
            "DICT_7X7_250": cv2.aruco.DICT_7X7_250,
            "DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
            "DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL
        }
        self.card_img = cv2.imread("cartasPoker/2_of_clubs.png")
        self.tvec = None
        self.rvec = None
        self._arucoInit()


    def _arucoInit(self):
        # Check that we have a valid ArUco marker
        if self.ARUCO_DICT.get(self.desired_aruco_dictionary, None) is None:
            print("[INFO] ArUCo tag of '{}' is not supported".format(
                self.desired_aruco_dictionary))
            return

        # Load the ArUco dictionary
        print("[INFO] detecting '{}' markers...".format(
            self.desired_aruco_dictionary))
        self.this_aruco_dictionary = cv2.aruco.Dictionary_get(self.ARUCO_DICT[self.desired_aruco_dictionary])
        self.this_aruco_parameters = cv2.aruco.DetectorParameters_create()

        # Camera calibration parameters (replace with your camera's values)
        self.camera_matrix = np.array([[1000, 0, 320],
                                [0, 1000, 240],
                                [0, 0, 1]], dtype=float)
        self.dist_coeffs = np.zeros((4, 1))


    def drawCards(self, frame, dealerHandList, playerHandList, showResult=False):
        # Detect ArUco markers in the video frame
        corners, ids, rejected = cv2.aruco.detectMarkers(
            frame, self.this_aruco_dictionary, parameters=self.this_aruco_parameters)

        if len(corners) > 0:
            for i in range(0, len(ids)):
                self.rvec, self.tvec, markerPoints = cv2.aruco.estimatePoseSingleMarkers(
                    corners[i], 0.025, self.camera_matrix, self.dist_coeffs)
                

                #cv2.aruco.drawDetectedMarkers(frame, corners)
                #cv2.drawFrameAxes(frame, self.camera_matrix, self.dist_coeffs, self.rvec, self.tvec, 0.01)


                frame = self._drawDealerHand(dealerHandList, frame)
                frame = self._drawPlayerHands(playerHandList, frame)

                if showResult:
                    frame = self.show_result(frame, dealerHandList, playerHandList)

                #frame = self._add_text_on_aruco(frame, str(13), self.rvec, self.tvec, self.camera_matrix, self.dist_coeffs, 0, 0)

        return frame


    def _drawDealerHand(self, dealerHandList, frame):
        numCards = len(dealerHandList)

        #separacion de tamaño de una carta horizontal = 0.02
        #separacion de tamaño de una carta vertical = 0.029

        separacion_horizontal = 0.0007
        separacion_vertical = 0.01

        if numCards%2 == 0:
            inicio_offset_x = -(separacion_horizontal/2) - (numCards/2) * 0.02 - (numCards/2 - 1) * separacion_horizontal
        else:
            inicio_offset_x = -0.01 - (numCards//2) * 0.02 - (numCards//2) * separacion_horizontal

        for card in dealerHandList:
            if card.isVisible:
                image = cv2.imread(card.image)
            else:
                image = cv2.imread("cartasEspeciales/back_card.png")
            frame = self._add_image_on_aruco(frame, image, self.camera_matrix, self.dist_coeffs, inicio_offset_x, separacion_vertical)
            inicio_offset_x += separacion_horizontal + 0.02
        
        numValueHand, _ = self._calculate_hand_value(dealerHandList)
        frame = self._add_text_on_aruco(frame, numValueHand, self.camera_matrix, self.dist_coeffs, self._dist_x_num(numValueHand), separacion_vertical + 0.029)
        
        return frame
    

    def _drawPlayerHands(self, playerHandsList, frame):
        numHands = len(playerHandsList)

        #separacion de tamaño de una carta horizontal = 0.02
        #separacion de tamaño de una carta vertical = 0.029

        dist_hands = 0.08

        if numHands%2 == 0:
            center = -(numHands/2)*dist_hands + (dist_hands/2)
        else:
            center = -(numHands//2)*dist_hands


        for hand in playerHandsList:
            frame = self._drawPlayerHand(center, hand, frame)
            numValueHand, _ = self._calculate_hand_value(hand)
            frame = self._add_text_on_aruco(frame, numValueHand, self.camera_matrix, self.dist_coeffs, center+self._dist_x_num(numValueHand), -0.01)
            center += dist_hands 
        return frame

        

    def _drawPlayerHand(self, centro, currentHand, frame):
        numCards = len(currentHand)

        separacion_horizontal = 0.0007
        separacion_vertical = -0.01-0.029

        if numCards%2 == 0:
            inicio_offset_x = -(separacion_horizontal/2) - (numCards/2) * 0.02 - (numCards/2 - 1) * separacion_horizontal + centro
        else:
            inicio_offset_x = -0.01 - (numCards//2) * 0.02 - (numCards//2) * separacion_horizontal + centro

        for card in currentHand:
            image = cv2.imread(card.image)
            frame = self._add_image_on_aruco(frame, image, self.camera_matrix, self.dist_coeffs, inicio_offset_x, separacion_vertical)
            inicio_offset_x += separacion_horizontal + 0.02
        
        return frame


    
    def _add_image_on_aruco(self, frame, card_img, camera_matrix, dist_coeffs, offset_x, offset_y, scale=1, angle=0, flip_vertical=True):
        if flip_vertical:
            card_img = cv2.flip(card_img, 0)
        
        # Obtener dimensiones originales
        card_h, card_w = card_img.shape[:2]
        new_card_w = int(card_w * scale)
        new_card_h = int(card_h * scale)
        
        # 3. Redimensionar usando un mejor método de interpolación
        #card_img_resized = cv2.resize(card_img, (new_card_w, new_card_h), interpolation=cv2.INTER_LANCZOS4)
        
        scale_factor = 0.00004

        object_points = np.array([
            [0 + offset_x, 0 + offset_y, 0],
            [new_card_w * scale_factor + offset_x, 0 + offset_y, 0],
            [new_card_w * scale_factor + offset_x, new_card_h * scale_factor + offset_y, 0],
            [0 + offset_x, new_card_h * scale_factor + offset_y, 0]
        ], dtype=np.float32)

        # Project the 3D points into 2D image space
        img_points, _ = cv2.projectPoints(object_points, self.rvec, self.tvec, camera_matrix, dist_coeffs)

        # Get the 2D points as integers
        img_points = np.int32(img_points).reshape((-1, 2))

        # Create a mask for the card and warp the image onto the frame
        frame_h, frame_w = frame.shape[:2]
        mask = np.zeros((frame_h, frame_w), dtype=np.uint8)
        cv2.fillConvexPoly(mask, img_points, 255)

        # Warp the card image onto the frame with better interpolation
        warp_matrix = cv2.getPerspectiveTransform(
            np.float32([[0, 0], [new_card_w, 0], [new_card_w, new_card_h], [0, new_card_h]]),
            img_points.astype(np.float32)
        )
        warped_card = cv2.warpPerspective(card_img, warp_matrix, (frame_w, frame_h),
                                        flags=cv2.INTER_LANCZOS4)

        # Combine the warped card with the frame
        frame = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(mask))
        frame = cv2.add(frame, warped_card)

        return frame
    
    def _add_text_on_aruco(self, frame, text, camera_matrix, dist_coeffs, offset_x, offset_y, scale=1, angle=0, color=(255, 255, 255, 255)):
        # Crear una imagen en blanco para el texto
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 1 * scale
        thickness = 2
        
        # Obtener el tamaño del texto
        (text_w, text_h), baseline = cv2.getTextSize(text, font, font_scale, thickness)
        
        # Crear una imagen en blanco con un pequeño padding
        padding = 10
        text_img = np.zeros((text_h + 2*padding, text_w + 2*padding, 4), dtype=np.uint8)
        
        # Agregar el texto a la imagen
        cv2.putText(text_img, text, 
            (padding, text_h + padding - baseline), 
            font, font_scale+0.02, (0, 0, 0, 255), thickness + 3)

        cv2.putText(text_img, text, 
                    (padding, text_h + padding - baseline), 
                    font, font_scale, color, thickness)

        # Rotar 180 grados y voltear horizontalmente para corregir la orientación
        text_img = cv2.rotate(text_img, cv2.ROTATE_180)
        text_img = cv2.flip(text_img, 1)

        # Convertir dimensiones a metros (usando el mismo factor de escala)
        scale_factor = 0.0003
        text_h_meters = (text_h + 2*padding) * scale_factor
        text_w_meters = (text_w + 2*padding) * scale_factor

        # Definir los puntos 3D para el texto
        object_points = np.array([
            [0 + offset_x, 0 + offset_y, 0],
            [text_w_meters + offset_x, 0 + offset_y, 0],
            [text_w_meters + offset_x, text_h_meters + offset_y, 0],
            [0 + offset_x, text_h_meters + offset_y, 0]
        ], dtype=np.float32)

        # Proyectar los puntos 3D al espacio 2D
        img_points, _ = cv2.projectPoints(object_points, self.rvec, self.tvec, camera_matrix, dist_coeffs)
        img_points = np.int32(img_points).reshape((-1, 2))

        # Crear máscara y aplicar transformación perspectiva
        frame_h, frame_w = frame.shape[:2]
        mask = np.zeros((frame_h, frame_w), dtype=np.uint8)
        cv2.fillConvexPoly(mask, img_points, 255)

        # Matriz de transformación perspectiva
        warp_matrix = cv2.getPerspectiveTransform(
            np.float32([[0, 0], [text_w + 2*padding, 0], 
                    [text_w + 2*padding, text_h + 2*padding], 
                    [0, text_h + 2*padding]]),
            img_points.astype(np.float32)
        )

        # Aplicar la transformación perspectiva
        warped_text = cv2.warpPerspective(text_img, warp_matrix, (frame_w, frame_h),
                                        flags=cv2.INTER_LINEAR)

        # Combinar el texto transformado con el frame
        alpha_mask = warped_text[:, :, 3] / 255.0
        alpha_mask = np.stack([alpha_mask] * 3, axis=-1)
        warped_text_rgb = warped_text[:, :, :3]
        
        frame = frame * (1 - alpha_mask) + warped_text_rgb * alpha_mask

        return frame.astype(np.uint8)
    
    
    def _calculate_hand_value(self, hand):
        """Calcula el valor de una mano teniendo en cuenta los ases"""
        value = 0
        valueMax = 0
        aces = False

        result = ""
        
        for card in hand:
            if not card.isVisible:
                continue
            if card.value in ['jack', 'queen', 'king']:
                value += 10
            elif card.value == 'ace':
                aces = True
                value += 1
            else:
                value += int(card.value)
        
        if aces and value + 10 <= 21:
            valueMax = 10 + value
            result = f"{value}/{valueMax}"
        else:
            result = str(value)

        if valueMax == 21:
            result = "21"

        if valueMax != 0:
            value = valueMax

        return result, value
    

    def _dist_x_num(self, value):
        match len(value):
            case 1:
                return -0.006
            case 2:
                return -0.01
            case 4:
                return -0.016
            case _:
                return -0.018
    
    def show_result(self, frame, dealerHand, playerHands):
        _, dealerHandValue = self._calculate_hand_value(dealerHand)

        separacion_vertical = -0.033

        numHands = len(playerHands)
        dist_hands = 0.08
        if numHands%2 == 0:
            center = -(numHands/2)*dist_hands + (dist_hands/2)
        else:
            center = -(numHands//2)*dist_hands

        center -= 0.014

        for hand in playerHands:
            _, playerHandValue = self._calculate_hand_value(hand)

            if playerHandValue > 21:
                frame = self._add_text_on_aruco(frame, "Bust", self.camera_matrix, self.dist_coeffs, center, separacion_vertical, color=(0, 0, 255, 255))
            elif dealerHandValue > 21:
                frame = self._add_text_on_aruco(frame, "Win", self.camera_matrix, self.dist_coeffs, center+0.002, separacion_vertical, color=(0, 255, 0, 255))
            elif playerHandValue > dealerHandValue:
                frame = self._add_text_on_aruco(frame, "Win", self.camera_matrix, self.dist_coeffs, center+0.002, separacion_vertical, color=(0, 255, 0, 255))
            elif playerHandValue < dealerHandValue:
                frame = self._add_text_on_aruco(frame, "Lose", self.camera_matrix, self.dist_coeffs, center, separacion_vertical, color=(0, 0, 255, 255))
            elif playerHandValue == dealerHandValue:
                frame = self._add_text_on_aruco(frame, "Push", self.camera_matrix, self.dist_coeffs, center, separacion_vertical, color=(0, 127, 255, 255))
            center += dist_hands
                
                
        return frame

In [5]:
def print_game_state(game):
    """Imprime el estado actual del juego"""
    state = game.get_game_state()
    
    print("\n" + "="*50)
    print("ESTADO DEL JUEGO:")
    print(f"Fichas: {state['player_chips']} | Apuesta actual: {state['current_bet']}")
    
    # Mostrar cartas del dealer
    print("\nCartas del Dealer:")
    if state['game_state'] == GameState.PLAYER_TURN:
        # Durante el turno del jugador, solo mostrar la primera carta del dealer
        print(f"Carta visible: {state['dealer_hand'][0].value} de {state['dealer_hand'][0].suit}")
        print("Carta oculta: ???")
    else:
        dealer_cards = [f"{card.value} de {card.suit}" for card in state['dealer_hand']]
        print(f"Cartas: {', '.join(dealer_cards)}")
        print(f"Valor total: {game.calculate_hand_value(state['dealer_hand'])}")
    
    # Mostrar cartas del jugador
    print("\nTus manos:")
    for i, hand in enumerate(state['player_hands']):
        cards = [f"{card.value} de {card.suit}" for card in hand]
        value = game.calculate_hand_value(hand)
        current = " (Mano actual)" if i == state['current_hand_index'] and state['game_state'] == GameState.PLAYER_TURN else ""
        print(f"Mano {i+1}{current}: {', '.join(cards)} (Valor: {value})")
    
    # Mostrar acciones disponibles
    if state['game_state'] == GameState.PLAYER_TURN:
        print("\nAcciones disponibles:")
        print("- Pedir carta (índice arriba)")
        print("- Plantarse (puño cerrado)")
        if len(state['player_hands'][state['current_hand_index']]) == 2:
            print("- Doblar (índice y medio arriba)")
        if state['can_split']:
            print("- Dividir (pulgar e índice arriba)")
    
    print("="*50 + "\n")

In [ ]:
def main():
    """
    Main method of the program.
    """
    deckShowAruco = DeckShowAruco()                                                 # Inicializar la clase para detectar aruco y mostrar cartas
    game = BlackjackGame()                                                          # Inicializar el juego
    gesture_recognizer = GestureRecognizer()                                        # Inicializar el reconocedor de movimientos
    action_cooldown = 0
    
    print("\n¡Bienvenido al Blackjack!")
    print(f"Tienes {game.player_chips} fichas.")
    success = game.place_bet(100)  # Apuesta inicial fija de 100
    if not success:
        print("Error al colocar la apuesta inicial")
        return
    print("Apuesta inicial de 100 fichas colocada.")
    
    cap = cv2.VideoCapture(0)
    
    with gesture_recognizer.mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        
        print_game_state(game)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_rgb)
            
            current_action = BlackjackAction.NONE
            
            if results.multi_hand_landmarks and game.game_state == GameState.PLAYER_TURN:
                for hand_landmarks in results.multi_hand_landmarks:
                    current_action = gesture_recognizer.detect_gesture(hand_landmarks)          # Detectar gestos y acciones
                    
                    num_fingers_up, fingers_up, is_right_hand = gesture_recognizer.count_fingers_up(hand_landmarks)

                    # Dibujar información de depuración
                    gesture_recognizer.draw_debug_info(frame, current_action, hand_landmarks, fingers_up, is_right_hand)
                    # Procesar acción si es diferente a la última y no hay cooldown
                    if (current_action != BlackjackAction.NONE and action_cooldown == 0):
                        success, message = game.process_action(current_action)
                        if success:
                            print(f"\n{message}")
                            print_game_state(game)
                            action_cooldown = 15  # Esperar 15 frames antes de aceptar otra acción
            
            frame = deckShowAruco.drawCards(frame, game.dealer_hand, game.player_hands)

            # Reducir el cooldown
            if action_cooldown > 0:
                action_cooldown -= 1
            
            # Si el juego ha terminado, mostrar resultados y reiniciar
            if game.game_state == GameState.GAME_OVER:
                print("\n¡Juego terminado!")
                print(f"Tus fichas: {game.player_chips}")
                
                # Preguntanbr si quiere jugar otra mano
                if game.player_chips >= 100:  # Asegurar que tiene suficientes fichas
                    game = BlackjackGame()
                    game.player_chips = game.player_chips  # Mantener las fichas ganadas
                    game.place_bet(100)
                    print("\nNueva mano comenzada con apuesta de 100 fichas.")
                    print_game_state(game)
                else:
                    print("\n¡Te has quedado sin fichas! Fin del juego.")
                    break
            
            cv2.imshow('Blackjack AR', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    print(__doc__)
    main()

In [ ]:
def main():
    """
    Main method of the program.
    """
    deckShowAruco = DeckShowAruco()                                                 # Inicializar la clase para detectar aruco y visualizar cartas
    game = BlackjackGame()                                                          # Inicializar el juego
    gesture_recognizer = GestureRecognizer()                                        # Inicializar el reconocedor de movimientos
    action_cooldown = 0
    show_result = True
    
    print("\n¡Bienvenido al Blackjack!")
    print(f"Tienes {game.player_chips} fichas.")
    success = game.place_bet(100)  # Apuesta inicial fija de 100
    if not success:
        print("Error al colocar la apuesta inicial")
        return
    print("Apuesta inicial de 100 fichas colocada.")
    
    cap = cv2.VideoCapture(0)
    
    with gesture_recognizer.mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        
        print_game_state(game)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_RGB)
            
            current_action = BlackjackAction.NONE

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:

                    if game.game_state == GameState.PLAYER_TURN:
                        current_action = gesture_recognizer.detect_gesture(hand_landmarks)          # Detectar gestos y acciones
                        if (current_action != BlackjackAction.NONE and action_cooldown == 0):
                            success, message = game.process_action(current_action)
                            if success:
                                print(f"\n{message}")
                                print_game_state(game)
                                action_cooldown = 15  # Esperar 15 frames antes de aceptar otra acción
            
            if game.game_state == GameState.PLAYER_TURN:
                frame = deckShowAruco.drawCards(frame, game.dealer_hand, game.player_hands)

            # Reducir el cooldown
            if action_cooldown > 0:
                action_cooldown -=1
            
            # Si el juego ha terminado, mostrar resultados y reiniciar
            if game.game_state == GameState.GAME_OVER:
                if show_result:
                    print("\n¡Juego terminado!")
                    print(f"Tus fichas: {game.player_chips}")
                    show_result = False


                # Enter para jugar otra mano
                if cv2.waitKey(1) & 0xFF == 13:
                    if game.player_chips >= 100:                # Asegurar que tiene suficientes fichas
                        chips = game.player_chips
                        game = BlackjackGame()
                        game.player_chips = chips               # Mantener las fichas ganadas
                        game.place_bet(100)
                        print("\nNueva mano comenzada con apuesta de 100 fichas.")
                        print_game_state(game)
                        show_result = True
                    else:
                        print("\n¡Te has quedado sin fichas! Fin del juego.")
                        break
                    
            cv2.imshow('Blackjack AR', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    print(__doc__)
    main()

In [6]:
from enum import Enum
import cv2
import numpy as np
import time

class BettingState(Enum):
    WAITING = 0
    HOVERING = 1
    SELECTED = 2
    CONFIRMED = 3

class ChipValue(Enum):
    CHIP_100 = 100
    CHIP_500 = 500
    CHIP_1000 = 1000
    CHIP_5000 = 5000
    ALL_IN = -1  # Valor especial para all-in

class BettingSystem:
    def __init__(self, initial_balance=10000):
        self.state = BettingState.WAITING
        self.current_bet = 0
        self.balance = initial_balance
        self.hover_start_time = 0
        self.hover_duration = 1.0  # Segundos necesarios para confirmar
        self.last_hovered_chip = None
        
        # Definir posiciones de las fichas en la pantalla
        self.chip_positions = {
            ChipValue.CHIP_100: (50, 50, 100, 100),   # x, y, width, height
            ChipValue.CHIP_500: (160, 50, 100, 100),
            ChipValue.CHIP_1000: (270, 50, 100, 100),
            ChipValue.CHIP_5000: (380, 50, 100, 100),
            ChipValue.ALL_IN: (490, 50, 100, 100)
        }

    def process_hand_position(self, hand_landmarks, frame):
        """
        Procesa la posición de la mano y actualiza el estado de la apuesta
        """
        current_time = time.time()  # Usar time.time() para el tiempo actual
        
        # Obtener posición del dedo índice (landmark 8)
        index_finger = hand_landmarks.landmark[8]
        screen_x = int(index_finger.x * frame.shape[1])
        screen_y = int(index_finger.y * frame.shape[0])
        
        # Verificar si el dedo está sobre alguna ficha
        hovered_chip = None
        for chip, (x, y, w, h) in self.chip_positions.items():
            if x <= screen_x <= x + w and y <= screen_y <= y + h:
                hovered_chip = chip
                break

        # Máquina de estados
        if self.state == BettingState.WAITING:
            if hovered_chip is not None:
                self.state = BettingState.HOVERING
                self.hover_start_time = current_time
                self.last_hovered_chip = hovered_chip
        
        elif self.state == BettingState.HOVERING:
            if hovered_chip != self.last_hovered_chip:
                self.state = BettingState.WAITING
            elif current_time - self.hover_start_time >= self.hover_duration:
                self.state = BettingState.SELECTED
                self._process_chip_selection(hovered_chip)
        
        elif self.state == BettingState.SELECTED:
            self.state = BettingState.CONFIRMED
        
        return self.state

    def _process_chip_selection(self, chip):
        """
        Procesa la selección de una ficha y actualiza la apuesta
        """
        if chip == ChipValue.ALL_IN:
            self.current_bet = self.balance
        else:
            chip_value = chip.value
            if self.current_bet + chip_value <= self.balance:
                self.current_bet += chip_value

    def draw_betting_ui(self, frame):
        """
        Dibuja la interfaz de usuario para las apuestas
        """
        # Dibujar cada ficha
        for chip, (x, y, w, h) in self.chip_positions.items():
            # Color base para las fichas
            color = (0, 100, 200)  # Azul por defecto
            
            # Si está siendo hover, cambiar color
            if self.state == BettingState.HOVERING and chip == self.last_hovered_chip:
                color = (0, 255, 0)  # Verde para hover
            
            # Dibujar círculo de la ficha
            cv2.circle(frame, (x + w//2, y + h//2), w//2, color, -1)
            
            # Texto de valor
            text = "ALL IN" if chip == ChipValue.ALL_IN else str(chip.value)
            text_size = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
            text_x = x + (w - text_size[0])//2
            text_y = y + (h + text_size[1])//2
            cv2.putText(frame, text, (text_x, text_y),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

        # Mostrar apuesta actual y balance
        cv2.putText(frame, f"Bet: ${self.current_bet}", (10, frame.shape[0] - 60),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(frame, f"Balance: ${self.balance}", (10, frame.shape[0] - 20),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    def reset_bet(self):
        """
        Reinicia el sistema de apuestas
        """
        self.state = BettingState.WAITING
        self.current_bet = 0
        self.last_hovered_chip = None

    def get_current_bet(self):
        """
        Retorna la apuesta actual
        """
        return self.current_bet

# Prueba final

In [16]:
def main():
    """
    Main method of the program.
    """
    deckShowAruco = DeckShowAruco()                                                 # Inicializar la clase para detectar aruco y visualizar cartas
    game = BlackjackGame()                                                          # Inicializar el juego
    gesture_recognizer = GestureRecognizer()                                        # Inicializar el reconocedor de movimientos
    betting_system = BettingSystem() 
    action_cooldown = 0
    show_result = True
    




    print("\n¡Bienvenido al Blackjack!")
    print(f"Tienes {game.player_chips} fichas.")
    
    
    cap = cv2.VideoCapture(0)
    
    with gesture_recognizer.mp_hands.Hands(
            model_complexity=0,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as hands:
        
        #print_game_state(game)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = hands.process(frame_RGB)
            
            current_action = BlackjackAction.NONE


            if game.game_state == GameState.WAITING_FOR_BET: 
                betting_system.draw_betting_ui(frame)

            if results.multi_hand_landmarks:
                for hand_landmarks in results.multi_hand_landmarks:

                    if game.game_state == GameState.WAITING_FOR_BET:
                        bet_state = betting_system.process_hand_position(hand_landmarks, frame)
                        if bet_state == BettingState.CONFIRMED:
                            amount = betting_system.get_current_bet()
                            success = game.place_bet(amount)
                            if not success:
                                print("Error al colocar la apuesta inicial")
                            print(f"Apuesta inicial de {amount} fichas colocada.")

                    elif game.game_state == GameState.PLAYER_TURN:
                        current_action = gesture_recognizer.detect_gesture(hand_landmarks)          # Detectar gestos y acciones
                        
                        if (current_action != BlackjackAction.NONE and action_cooldown == 0):
                            success, message = game.process_action(current_action)
                            if success:
                                print(f"\n{message}")
                                print_game_state(game)
                                action_cooldown = 15  # Esperar 15 frames antes de aceptar otra acción
            
            if game.game_state == GameState.PLAYER_TURN:
                frame = deckShowAruco.drawCards(frame, game.dealer_hand, game.player_hands)

            # Reducir el cooldown
            if action_cooldown > 0:
                action_cooldown -=1
            
            # Si el juego ha terminado, mostrar resultados y reiniciar
            if game.game_state == GameState.GAME_OVER:
                frame = deckShowAruco.drawCards(frame, game.dealer_hand, game.player_hands, True)
                if show_result:
                    print("\n¡Juego terminado!")
                    print(f"Tus fichas: {game.player_chips}")
                    show_result = False


                # Enter para jugar otra mano
                if cv2.waitKey(1) & 0xFF == 13:
                    if game.player_chips >= 100:                # Asegurar que tiene suficientes fichas
                        chips = game.player_chips
                        game = BlackjackGame()
                        game.player_chips = chips               # Mantener las fichas ganadas
                        game.place_bet(100)
                        print("\nNueva mano comenzada con apuesta de 100 fichas.")
                        print_game_state(game)
                        show_result = True
                    else:
                        print("\n¡Te has quedado sin fichas! Fin del juego.")
                        break
                    
            cv2.imshow('Blackjack AR', frame)
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    print(__doc__)
    main()

Automatically created module for IPython interactive environment
[INFO] detecting 'DICT_4X4_100' markers...

¡Bienvenido al Blackjack!
Tienes 2000 fichas.
Apuesta inicial de 500 fichas colocada.

Te plantas

ESTADO DEL JUEGO:
Fichas: 2500 | Apuesta actual: 500

Cartas del Dealer:
Cartas: 7 de diamonds, queen de spades
Valor total: 17

Tus manos:
Mano 1: queen de hearts, jack de diamonds (Valor: 20)


¡Juego terminado!
Tus fichas: 2500

Nueva mano comenzada con apuesta de 100 fichas.

ESTADO DEL JUEGO:
Fichas: 2400 | Apuesta actual: 100

Cartas del Dealer:
Carta visible: ace de clubs
Carta oculta: ???

Tus manos:
Mano 1 (Mano actual): 10 de diamonds, jack de hearts (Valor: 20)

Acciones disponibles:
- Pedir carta (índice arriba)
- Plantarse (puño cerrado)
- Doblar (índice y medio arriba)
- Dividir (pulgar e índice arriba)


Te plantas

ESTADO DEL JUEGO:
Fichas: 2600 | Apuesta actual: 100

Cartas del Dealer:
Cartas: ace de clubs, 7 de spades
Valor total: 18

Tus manos:
Mano 1: 10 de diam